# PQ

PQ（Product Quantization，乘积量化）是一种高效的向量压缩与近似距离计算方法

PQ 的核心思想是：

将高维向量拆分为多个子向量，在子空间内进行独立量化(使用中心节点向量代替实际向量)，再通过查表法快速计算近似距离

## PQ的核心优势

- 高压缩率：将浮点向量压缩为整数索引，存储占用大幅下降
- 高效距离计算：通过查表法快速估算距离，降低计算量
- 可组合性强：可与 IVF、HNSW 等索引结构结合，实现高效大规模检索；
- 近似精度可调：通过调整子向量数m 和 聚类中心数k 平衡精度与存储。

In [4]:
import numpy as np
import matplotlib as plt
import time
from scipy.cluster.vq import kmeans2, vq

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)

In [ ]:
class ProductQuantization:
    '''
    乘积量化算法实现
    '''
    def __init__(self, M=8, K=256) -> None:
        '''
        M: 子空间数量（将原始向量分割成多少段）
        K: 每个子空间的聚类中心数量（必须是2的幂次）
        '''
        self.M = M
        self.K = K
        self.codebooks = None # 码本: 存储每个子空间的聚类中心 shape(M, K, sub_dim) M个子空间，每个子空间K个质心，每个质心sub_dim维度
        self.sub_dim = None   # 每个子空间的维度
        self.is_trained = False

    def train(self, vectors, max_iters=100):
        '''
        训练PQ的码本
        vectors: 训练数据，shape(n_vectors, dim)
        max_iter: K-means最大迭代次数
        '''
        n_vectors, dim = vectors.shape

        if dim % self.M != 0:
            raise ValueError(f'vector dimension:{dim} can not be divisible by M:{self.M}')

        self.sub_dim = dim // self.M
        self.codebooks = np.zeros((self.M, self.K, self.sub_dim), dtype=np.float32)

        print(f'Start training PQ codebooks: the {dim}-dimension vector is partitioned into {self.M} subspaces, each subspace has {self.sub_dim}-dimensional.')
        print(f"Each subspace uses K={self.K} cluster centers")

        for m in range(self.M):
            print(f'Training subspace {m+1}/{self.M}...')

            # 提取当前子空间数据
            sub_vectors = vectors[:, m*self.sub_dim:(m+1)*self.sub_dim]
            
            # 使用K-means聚类
            # kmeans2返回聚类中心和每个点所属的簇标签
            centroids, labels = kmeans2(sub_vectors, self.K, iter=max_iters, minit='points') # minit='points' 表示从数据点中随机选择 K 个点作为初始质心
            self.codebooks[m] = centroids.astype(np.float32)
            
        self.is_trained = True
        print('PQ codebook training completed!')    
        return self.codebooks

    def encode(self, vectors):
        '''
        将向量编码为PQ码

        参数:
        - vectors: 待编码的向量，shape(n_vectors, dim)
        
        返回:
        - codes: PQ编码，shape(n_vectors, M) 每个元素都是[0,K-1]的整数
        '''
        if not self.is_trained:
            raise ValueError('Please Train PQ Codebook')

        n_vectors = vectors.shape[0]
        codes = np.zeros((n_vectors, self.M), dtype=np.uint8)

        for m in range(self.M):
            # 提取当前子空间的向量
            sub_vectors = vectors[:, m*self.sub_dim: (m+1)*self.sub_dim] # shape(n_vectors, sub_dim)

            # 为每个子向量找到最近的聚类中心
            # labels：每个子向量对应的最近聚类中心索引（0 到 K-1 的整数），shape 为 (n_vectors,1)
            labels, _ = vq(sub_vectors, self.codebooks[m]) # self.codebooks[m] 第m个子空间的K个质心，shape(K, sub_dim)
            codes[:, m] = labels

        return codes

    def decode(self, codes):
        '''
        将PQ码解码为近似向量
        参数:
        - codes: PQ编码 shape(n_vectors, M)
        返回:
        - approx_vectors: 近似向量，shape(n_vectors, dim) dim=sub_dim*M
        '''
        if not self.is_trained:
            raise ValueError('Please Train PQ Codebook')

        n_vectors = codes.shape[0]
        dim = self.M * self.sub_dim
        approx_vectors = np.zeros((n_vectors, dim), dtype=np.float32)

        for m in range(self.M):
            # 使用聚类中心向量的值代替编码
            approx_vectors[:, m*self.sub_dim:(m+1)*self.sub_dim] = \
                self.codebooks[m][codes[:, m]]

        return approx_vectors
 